# [이론5] Batch Normalization and Dropout



---

## 학습 목표
- 배치정규화 (Batch Normalization) 의 개념과 배치정규화를 통해 얻는 효과를 이해합니다.
- 드롭아웃 (Dropout) 의 필요성과 특징을 파악합니다.


------

## 목차

#### 1. Batch Normalization (배치 정규화)
#### 2. Dropout
#### 3. Tensorflow에서 Batch Normalization과 Dropout 사용하기

----

## 1. Batch Normalization (배치 정규화)

**논문**: [Batch Normalization: Accelerating Deep Network Training by Reducing Internal Covariate Shift (ICML 2015)](https://arxiv.org/abs/1502.03167)

### 1-1. Batch와 Normalization

이제까지 학습한 내용을 바탕으로 딥러닝 모델을 직접 학습시켰다면, batch(배치)와 normalization(정규화)의 개념은 익숙하실 것이라 생각합니다. 간단히 이 둘에 대해 다시 짚고 넘어가도록 하겠습니다.

배치란 데이터셋의 전체 데이터 내에서 모델 학습 한번에 적용하기 위해 뽑아낸 데이터의 일부를 의미합니다.

<center><img src="./img/batch-size.png" width=600></center>

배치를 이용하여 학습을 하는 이유에는 여러가지가 있습니다. 그 중 대표적인 이유는, 데이터셋의 크기가 일반적으로 매우 크기 때문에 이를 한번에 모델에 적용할 수 없기 때문입니다. 그렇기에 전체 데이터를 사용하여 동작하는 gradient descent 알고리즘을 쉽게 적용할 수 없게 되었습니다. 따라서 데이터셋을 배치로 나누고 각 배치마다 gradient를 계산하여 학습하는 **Stochastic Gradient Descent(SGD)** 가 등장하게 되었습니다.

다음으로 **정규화(Normalization)** 란 입력하는 데이터가 가지는 feature들의 값을 모두 같은 범위 내로 변경하는 작업을 의미합니다. 일반적으로 대부분의 데이터는 각 feature 별로 값의 크기 범위가 다릅니다. 따라서, 이를 그대로 모델에 적용할 경우 수치 자체가 큰 값들을 가지는 feature들에 의해 모델 학습 과정이 영향을 받을 수 있습니다. 따라서 모든 feature들이 가지는 값의 범위를 균일하게 맞춰주면 **feature 별 중요도를 균일하게 맞추게 되어** 모두 공평하게 모델 학습에 기여할 수 있게 되는 것입니다. 이는 결국 모델이 데이터에 좀 더 **일반화(generalization)** 될 수 있다는 의미이기도 하며, **학습의 안정성을 높인다**는 의미이기도 합니다. <br> 또한 아래 그림에서도 확인할 수 있듯이, 정규화를 통해 값을 작은 값으로 scaling 해주면 **local optimum에 빠지더라도 더 쉽게 빠져나올 수 있기 때문에**, 더 높은 성능의 모델을 학습할 수 있게 됩니다.

<center><img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile5.uf.tistory.com%2Fimage%2F9996C5355CEB721A339D07"
alt="LeNet5" align="center" border="0" width="500" height=auto></center>

정규화를 수행하는 방법에는 크게 아래의 두가지가 있습니다.
1. **Min-Max Normalization** (최소-최대 정규화)
2. **Standardization** (표준화)

먼저 Min-Max Normalization은 각 feature의 최소값과 최대값의 차이를 구한 다음에 feature 내 모든 값들을 이 차이로 나누는 것입니다.

$$
x_{norm} = \frac{x - x_{min}}{x_{max} - x_{min}}
$$

이렇게 되면 정규화된 값은 모두 0과 1사이에 존재하게 됩니다.

다음으로 **Standardization**은 일반적으로 **표준화**라 불리기도 하는 정규화 방법입니다. 표준화는 데이터의 각 feature 별로 feature의 분포를 **표준** 정규 분포로 바꿔주기 위해 데이터의 평균(mean)과 표준편차(standard deviation, stdv)를 구하여 값을 scaling 하는 정규화 방법입니다.

$$ 
x_{norm} = \frac{x - \mu}{\sigma} \\
\text{where }\mu = \text{mean, } \sigma = \text{stdv}
$$

이때는 정규화된 값의 범위에는 여전히 제한이 없지만, 값들의 평균과 표준편차 값이 각각 0과 1이 되기 때문에 각 feature에 존재하는 값들이 대부분 비슷한 크기(scale)를 갖게 됩니다.

### 1-2. Batch Normalization 알고리즘

이제 배치와 정규화가 무엇인지 짚고 넘어갔으니, 이번 단원의 본론인 **Batch Normalization(배치 정규화)** 를 이야기하도록 하겠습니다. Batch Normalization이란 말 그대로 배치에 정규화를 적용하는 기법입니다. 구체적으로는 input이 하나의 layer와 그에 따르는 활성화 함수를 통과한 후에 나온 결과물에 정규화를 적용하는 것입니다. 이때 배치 정규화를 위해 구하는 평균과 표준편차 값은 하나의 배치 전체에서 각 차원별로 구하게 됩니다. 즉, $d$차원의 데이터 $m$개를 묶어서 하나의 배치를 구성했다면, 배치 정규화에서 구하는 평균과 표준편차는 $d$개가 된다고 할 수 있겠습니다.

$$
\text{Input } B = [(x_{1}^{1}, x_{1}^{2}, ..., x_{1}^{d}), (x_{2}^{1}, x_{2}^{2}, ..., x_{2}^{d}), ..., (x_{m}^{1}, x_{m}^{2}, ..., x_{m}^{d})]
$$

$$
\mu_{B}^{k} = \frac{1}{m} \sum_{i=1}^{m} x_{i}^{k} \text{,   } k=1, 2,..., d 
$$

$$
(\sigma_{B}^{k})^{2} = \frac{1}{m} \sum_{i=1}^{m} (x_{i}^{k} - \mu_{B}^{k})^{2} \text{,   } k=1, 2,..., d
$$

이제 한 배치의 각 차원별 평균과 표준편차 값을 구했으니 정규화를 수행할 수 있게 됐습니다. 이를 수행하는 식은 아래와 같습니다.
$$ 
\hat{x}_{i}^{k} = \frac{x_{i}^{k} - \mu_{B}^{k}}{\sqrt{(\sigma_{B}^{k})^{2} + \epsilon}} \text{,   } k=1, 2,..., d \text{,   } i=1, 2, ..., m 
$$

여기서 $\epsilon$은 0으로 나누기 문제를 방지하고자 도입한 매우 작은 실수입니다. 이 수식들을 그림으로 정리한 것은 아래와 같습니다. 좀 더 간결한 묘사를 위해 차원을 나타내는 $k$ 인덱스는 생략했습니다.

<center><img src="https://img1.daumcdn.net/thumb/R1280x0/?scode=mtistory2&fname=http%3A%2F%2Fcfile8.uf.tistory.com%2Fimage%2F99166C4B5BBDFFFA279D59" align="center" border="0" width="500" height=auto></center>

이렇게 하면 정규화 자체는 완료되었습니다. 하지만 논문에서 저자들은 이렇게만 정규화를 했을 때 추가적으로 문제가 생길 수 있다고 주장합니다. 예를 들어 정규화를 거치면 값들의 평균과 분산이 각각 0과 1이 됩니다. 그러나 이러한 값들을 Sigmoid 함수에 적용하면 대부분 값들이 0 근처에 모이게 되어 **사실상 선형(linear) 활성화 함수와 다름이 없어진다**는 것입니다.

<center><img src="./img/sigmoid-linear.png" width=300></center>

딥러닝 모델에서 비선형성이 사라지면 층을 여러개 쌓는 의미가 없어집니다. 이 문제를 해결하기 위하여 논문에서는 별도의 파라미터 $\gamma$와 $\beta$ 를 도입하여 배치 정규화 작업의 최종 결과물을 아래와 같이 정의합니다. 

$$ 
y_{i}^{k} = \gamma^{k} \hat{x}_{i}^{k} + \beta^{k} 
$$

여기서 $\gamma$와 $\beta$ 또한 차원별로 따로 존재하는 것에 유의하세요. 이 $\gamma$와 $\beta$는 **학습 가능한 파라미터**인데, 이는 대부분 딥러닝 모델이 배치를 이용한 SGD 기반의 학습법을 사용하기 때문입니다. 즉, 각 배치를 적용했을 때의 $\gamma$와 $\beta$는 전체 데이터를 한번에 적용했을 때 나올 것이라 기대되는 값의 **추정치**이기 때문에 이 또한 학습을 통해 알아내겠다는 것이 논문의 의도입니다.

$\gamma$ 는 정규화 이후 분포를 얼마나 스케일링 할지, 그리고 $\beta$ 는 정규화 이후 분포를 얼마나 이동할지를 나타냅니다.

이를 정리하여 하나의 알고리즘으로 표현하면, 다음 알고리즘과 같습니다. 간결한 묘사를 위해 차원을 나타내는 $k$ 인덱스를 생략했습니다.

<center><img src="https://blog.kakaocdn.net/dn/U4PdS/btqEgQFdPhS/mmAh3RQmKgUoAnjy87FZ11/img.png" align="center" border="0" width="400" height=auto></center>

### 1-3. Why Batch Normalization?

이렇게 하여 배치 정규화라는 것이 어떻게 작동하는 알고리즘인지 모두 알아보았습니다. 하지만 많은 분들이 여기까지 왔다면 한가지 드는 의문이 있을 것입니다. 아마도 **왜 배치 정규화를 적용하는 것인가**에 대한 의문이 생기셨으리라 생각합니다.

배치 정규화는 기존 딥러닝 모델의 구조에 영향을 주지 않은 채 추가 layer 삽입 만으로 적용할 수 있습니다. 즉, 배치 정규화라는 기법의 범용성이 훌륭하다는 의미입니다. 이와 같이 새로운 component를 적용함에 있어 기존 시스템의 변경이 필요 없는 성질을 **orthogonal**(수직적) 또는 **independent**(독립적) 하다고 합니다. 게다가 추가하기만 하는 것으로 큰 성능 향상을 이룰 수 있기 때문에 현재 딥러닝 모델에서 배치 정규화를 적용하지 않는 경우는 찾아보기 어려울 정도로 사실상의 표준 layer로 자리잡았습니다. 그렇다면 무엇 때문에 이러한 성능 향상을 가져올 수 있는 것일까요?

저자들은 논문에서 배치 정규화를 도입하게 된 계기(motivation)로 **Internal Covariance Shift**라는 것을 꼽습니다. 이는 각 layer의 활성화 함수를 통과할 때마다 출력되는 값의 분포가 서서히 바뀌는 현상을 말합니다. 아래 그림에서처럼 layer를 통과할 때마다 값의 분포가 바뀌는 것을 확인할 수 있습니다.

<center><img src="https://gaussian37.github.io/assets/img/dl/concept/batchnorm/4.png" align="center" border="0" width="600" height=auto></center>

이러한 변화 자체를 **Covariance Shift**라 합니다. 저자들은 이 현상이 layer 내에서 일어나기 때문에 Internal을 붙여서 묘사하고 있습니다. 이 현상은 layer 별로 입력되는 데이터의 분포가 모두 달라질 수 있기 때문에 학습이 불안정해지는 계기가 됩니다. 따라서 각 레이어 별로 정규화를 수행하게 되면 분포 자체를 항상 표준 정규 분포로 똑같이 맞출 수 있기 때문에 학습에도 긍정적인 영향을 주게 되는 것입니다.

지금까지의 내용을 통해 배치 정규화를 적용함으로써 얻을 수 있는 장점은 아래의 것들이 있습니다.

1. 학습률(learning rate)을 높게 설정할 수 있어 학습 속도를 빠르게 향상시킬 수 있습니다.
2. 학습을 할 때마다 각 층의 출력값을 정규화하기 때문에 가중치의 초기값 선택에 대한 의존성이 적어집니다.
3. 과적합(Overfitting)을 방지할 수 있습니다.
4. 활성화함수로 Sigmoid 함수나 tanh 함수를 사용하더라도 기울기 소실(Vanishing Gradient) 문제를 크게 개선할 수 있습니다.
5. Regularization 효과가 있습니다

하지만 이러한 배치 정규화도 미니 배치의 크기($B$)에 의존적이라는 점과 각 시점에서 통계치가 다른 RNN에서 적용하기 어렵다는 단점을 가지고 있습니다. 

## 2. Dropout

**논문1**: [ImageNet Classification with Deep Convolutional Neural Networks (NIPS 2012)](http://www.cs.toronto.edu/~kriz/imagenet_classification_with_deep_convolutional.pdf) (AlexNet 논문)

**논문2**: [Dropout: A Simple Way to Prevent Neural Networks from Overfitting](https://jmlr.org/papers/v15/srivastava14a.html)

이전 이론 (CNN Architecture)에서 AlexNet을 설명하면서 Dropout을 언급한 것을 기억하시나요? 그때도 간단히 설명했지만 여기서 좀더 구체적으로 짚고 넘어가도록 하겠습니다.

Dropout이란 **학습 단계**에서 인공 신경망 내에서 **일부 뉴런들의 연결을 삭제(drop)** 하는 것입니다. 연결이 끊긴 뉴런은 가중치 계산에 관여하지 않게 되면서 파라미터 수가 줄어드는 효과를 얻을 수 있게 됩니다. 이는 일반적으로 파라미터 수가 무지막지하게 늘어나는 Fully-connected layer에서 큰 영향을 발휘하며, 파라미터 수가 줄어드니 자연히 과적합(overfitting)이 생길 가능성도 낮아지게 됩니다. 즉, 정규화(Regularization)을 위해 사용하는 기법입니다.

Dropout은 layer 단위로 추가 가능하며, 만약 dropout이 추가된 layer의 경우 해당 layer에 있는 뉴런 중 일부를 확률적으로 비활성화시킵니다. 이때 각 뉴런이 비활성화 될 확률은 hyperparameter로, 일반적으로 $p$로 나타냅니다.<br> 비활성화 작업은 **모델 학습시에만 적용**됩니다. 그러므로 학습시 사용하는 모델의 파라미터 개수를 줄일 수 있게 되어 과적합을 방지하게 되는 것입니다. 반대로 테스트 시에는 Dropout을 적용하지 않습니다. <br> 테스트 시에 적용하지 않는다고 하여 **Dropout layer가 빠진 별도의 모델이 필요한 것은 아니고**, Tensorflow나 PyTorch 등의 딥러닝 라이브러리는 자동으로 테스트 시에 Dropout layer가 작동하지 않도록 설정해줍니다.

<center><img src="https://t1.daumcdn.net/cfile/tistory/2554714557F3814C09" align="center" border="0" width="500" height=auto></center>

Dropout이 처음 발표가 될 당시만 하더라도 Fully-connected layer에만 적용한 것으로 발표가 되었고, 이후 2014년에 초기 Dropout 개념을 좀 더 이론적으로 정립하고, convolutional layer에도 적용을 한 논문 “[Dropout: A Simple Way to Prevent Neural Networks from Overfitting](https://jmlr.org/papers/v15/srivastava14a.html)”이 발표되었습니다.

Dropout이 가지는 또 하나의 장점은 뉴런 선택이 매번 랜덤으로 이루어지기 때문에 서로 다른 여러개의 모델들을 합쳐서(Ensemble) 학습하는 듯한 효과를 준다는 것입니다.

<center><img src="./img/dropout-ensemble.png" width=500></center>

## 3. Tensorflow에서 Batch Normalization과 Dropout 사용하기

Tensorflow에서 배치 정규화와 Dropout을 적용하는 방법은 매우 간단합니다. 아래 두 줄의 코드를 통해 각 방법을 적용시킬 수 있습니다.

배치 정규화: `tf.layers.batch_normalization()`

Dropout: `tf.keras.layers.Dropout(rate = 0.3)`
- Dropout 코드에서 rate 인자는 사용자가 직접 설정하는 hyperparameter로, 제외되는 뉴런의 비율입니다 (0.3 = 30%의 뉴런을 랜덤하게 drop)

In [1]:
# tensorflow와 tf.keras를 임포트합니다
import tensorflow as tf
from tensorflow import keras

import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

fashion_mnist = keras.datasets.fashion_mnist
(train_X, train_y), (test_X, test_y) = fashion_mnist.load_data()

train_X = train_X / 255.0 
test_X = test_X / 255.0

train_X = train_X.reshape(-1, 28, 28, 1)
test_X = test_X.reshape(-1, 28, 28, 1)

# reshape 이후
print(train_X.shape, test_X.shape)

import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D
from tensorflow.keras.layers import Flatten, Dense, Dropout

2.4.0
4423680/4422102 [==============================] - 0s 0us/step
(60000, 28, 28, 1) (10000, 28, 28, 1)


In [2]:
model = keras.Sequential([

    keras.layers.InputLayer(input_shape=(28, 28, 1)),
    
    # Conv - BN - Activation - Pooling 순으로 Layer를 배치합니다.
    keras.layers.Conv2D(input_shape=(28, 28, 1),
                        filters=32,
                        kernel_size=4,
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2),

    # Conv - BN - Activation - Pooling 순으로 Layer를 배치합니다.
    keras.layers.Conv2D(filters=64,
                        kernel_size=[4, 4],
                        padding="same"),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.MaxPooling2D(pool_size=[2, 2], strides=2),

    keras.layers.Flatten(),
    keras.layers.Dense(128,
                       activation=keras.activations.relu,
                       kernel_initializer='he_uniform',
                       bias_initializer='he_uniform'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(10, activation=keras.activations.softmax)

])

print(model.summary())


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization (BatchNo (None, 28, 28, 1)         4         
_________________________________________________________________
conv2d (Conv2D)              (None, 28, 28, 32)        544       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 32)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 32)        128       
_________________________________________________________________
dropout (Dropout)            (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 64)        32832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 64)          0

----

<span style="color:rgb(120, 120, 120)">본 학습 자료를 포함한 사이트 내 모든 자료의 저작권은 엘리스에 있으며 외부로의 무단 복제, 배포 및 전송을 불허합니다.

Copyright @ elice all rights reserved</span>